<a href="https://colab.research.google.com/github/Maks2811/test_nagaev/blob/main/task_14_10_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=dac2536a02e4aa7d61c8270be674628b61ce84e6dc05db4804a6c9947e01333f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [35]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLMovieActors").getOrCreate()
df_movies = spark.read.csv("/content/movies.csv", header=True, inferSchema=True)
df_actors = spark.read.csv("/content/actors.csv", header=True, inferSchema=True)
df_movie_actors = spark.read.csv("/content/movie_actors.csv", header=True, inferSchema=True)

df_movies.printSchema()
df_actors.printSchema()
df_movie_actors.printSchema()

df_movies.show()
df_actors.show()
df_movie_actors.show()


# создаем временные таблицы
df_movies.createOrReplaceTempView("movies")
df_actors.createOrReplaceTempView("actors")
df_movie_actors.createOrReplaceTempView("movie_actors")

# соединяем таблицы
# join_df=spark.sql("""
# SELECT * FROM movies m
# JOIN movie_actors ma ON m.movie_id=ma.movie_id
# JOIN actors a ON ma.actor_id=a.actor_id
# """)

# join_df.show()

# 4.1 Найдите топ-5 жанров по количеству фильмов.

print("Результат 4.1")
spark.sql("""
SELECT genre, count(*) as num_movies
FROM movies
GROUP BY genre
ORDER BY num_movies DESC
LIMIT 5""").show()

# 4.2 Найдите актера с наибольшим количеством фильмов.

print("Результат 4.2")
spark.sql("""
SELECT name, num_movies
FROM
(SELECT name, count(title) as num_movies FROM actors a
JOIN movie_actors ma ON a.actor_id=ma.actor_id
JOIN movies m ON ma.movie_id=m.movie_id
GROUP BY name) as table_1
WHERE num_movies = (SELECT MAX(num_movies) FROM (SELECT name, count(title) as num_movies FROM actors a
JOIN movie_actors ma ON a.actor_id=ma.actor_id
JOIN movies m ON ma.movie_id=m.movie_id
GROUP BY name) as table_1)
""").show()

# 4.3 Подсчитайте средний бюджет фильмов по жанрам.

print("Результат 4.3")
spark.sql("""
SELECT genre, SUM(budget)/COUNT(*) as avg_budget
FROM movies
GROUP BY genre
""").show()

# 4.4 Найдите фильмы, в которых снялось более одного актера из одной страны.
print("Результат 4.4")
spark.sql("""
SELECT title, country, count(name) as num_actors
FROM actors a
JOIN movie_actors ma ON a.actor_id=ma.actor_id
JOIN movies m ON ma.movie_id=m.movie_id
GROUP by title, country
HAVING num_actors >1
""").show()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- budget: double (nullable = true)

root
 |-- actor_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country: string (nullable = true)

root
 |-- movie_id: integer (nullable = true)
 |-- actor_id: integer (nullable = true)

+--------+--------+------+------------+-------------+
|movie_id|   title| genre|release_date|       budget|
+--------+--------+------+------------+-------------+
|       1| Movie_1|Horror|  2000-12-31|8.660058311E7|
|       2| Movie_2|Comedy|  2001-12-31|1.274740083E7|
|       3| Movie_3|Action|  2002-12-31| 1.80157747E7|
|       4| Movie_4| Drama|  2003-12-31|4.817612061E7|
|       5| Movie_5| Drama|  2004-12-31| 7.40501611E7|
|       6| Movie_6|Action|  2005-12-31|1.476121831E7|
|       7| Movie_7| Drama|  2006-12-31|4.456703643E7|
|       8